In [4]:
# https://github.com/huggingface/notebooks/blob/master/examples/language_modeling_from_scratch.ipynb
# https://github.com/huggingface/transformers/tree/master/notebooks
# https://huggingface.co/transformers/model_doc/xlnet.html#transformers.XLNetTokenizer 
# https://colab.research.google.com/github/gmihaila/ml_things/blob/master/notebooks/pytorch/pretrain_transformers_pytorch.ipynb#scrollTo=VE2MRZZhd5uM 

In [5]:
import os
os.environ['HF_HOME'] = os.path.join(os.getcwd(), 'hf_cache')

from transformers import XLNetConfig, XLNetModel, XLNetTokenizer, XLNetLMHeadModel 
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import DataCollatorForPermutationLanguageModeling
from tqdm import tqdm

In [6]:
tokenizer = XLNetTokenizer(vocab_file='models/smiles_hiv_sp.model',
                           do_lower_case=False,
                           keep_accents=True
                           )

In [10]:
train_raw = load_dataset('csv', data_files=['data/ogb_molhiv/train_hiv.csv'])
test_raw = load_dataset('csv', data_files=['data/ogb_molhiv/test_hiv.csv'])
valid_raw = load_dataset('csv', data_files=['data/ogb_molhiv/valid_hiv.csv'])

Using custom data configuration default-0250d14726bc71f8
Reusing dataset csv (e:\molnlp\mol-prop\hf_cache\datasets\csv\default-0250d14726bc71f8\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)
100%|██████████| 1/1 [00:00<00:00, 76.95it/s]
Using custom data configuration default-d85d5e570e46467b
Reusing dataset csv (e:\molnlp\mol-prop\hf_cache\datasets\csv\default-d85d5e570e46467b\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)
100%|██████████| 1/1 [00:00<00:00, 71.45it/s]
Using custom data configuration default-e9711672e6359f2b
Reusing dataset csv (e:\molnlp\mol-prop\hf_cache\datasets\csv\default-e9711672e6359f2b\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)
100%|██████████| 1/1 [00:00<00:00, 62.49it/s]


In [27]:
tokenizer.special_tokens_map
#print(tokenizer('Clc1ccnc2c1ccc1c(Cl)ccnc12'))

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '<sep>',
 'pad_token': '<pad>',
 'cls_token': '<cls>',
 'mask_token': '<mask>',
 'additional_special_tokens': ['<eop>', '<eod>']}

In [32]:
tokenizer

<bound method XLNetTokenizer.get_special_tokens_mask of PreTrainedTokenizer(name_or_path='', vocab_size=1092, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='left', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<sep>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': ['<eop>', '<eod>']})>

In [15]:
test_id = 52
print(train_raw['train'][test_id]['smiles'])
#input_ids =tokenizer(train_raw['train'][test_id]['smiles'], padding='max_length', max_length=100)['input_ids'] 
input_ids =tokenizer(train_raw['train'][test_id]['smiles'])['input_ids'] 
print(input_ids)
print(tokenizer.convert_ids_to_tokens(input_ids))

CCOC(=O)C(=O)C1CCCCC1=O
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 53, 10, 7, 4, 3, 10, 7, 4, 3, 8, 70, 35, 7, 0, 0]
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<u

In [19]:

print(tokenizer.pad_token_id)
print(tokenizer.unk_token_id)

0
0


In [11]:
def tokenize_function_hiv(examples):
    out_dict = tokenizer(examples["smiles"])
    return out_dict

train_ds = train_raw.map(tokenize_function_hiv, batched=True, remove_columns=["smiles","HIV_active", "mol_id"])['train']
test_ds = test_raw.map(tokenize_function_hiv, batched=True, remove_columns=["smiles","HIV_active", "mol_id"])['train']
valid_ds = valid_raw.map(tokenize_function_hiv, batched=True, remove_columns=["smiles","HIV_active", "mol_id"])['train']

100%|██████████| 5/5 [00:00<00:00,  7.65ba/s]


In [7]:
n_layer = 4
model_config = XLNetConfig(
    vocab_size=tokenizer.vocab_size,
    n_layer=4,
    bi_data=True
)
model = XLNetLMHeadModel(model_config)

data_collator = DataCollatorForPermutationLanguageModeling(tokenizer=tokenizer)

In [8]:
max_samples = 2000000
batch_size = 70 
max_steps = max_samples // batch_size
save_steps = max_steps // 5
training_args = TrainingArguments(
    f"models/xlnet-smiles-s{max_samples}-b{batch_size}-l{n_layer}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    num_train_epochs=1,
    max_steps = max_steps,
    save_steps=save_steps,
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    data_collator=data_collator
)

max_steps is given, it will override any value given in num_train_epochs


In [10]:
trainer.train()

***** Running training *****
  Num examples = 6072715
  Num Epochs = 1
  Instantaneous batch size per device = 70
  Total train batch size (w. parallel, distributed & accumulation) = 70
  Gradient Accumulation steps = 1
  Total optimization steps = 28571
  2%|▏         | 500/28571 [08:45<8:10:39,  1.05s/it]

{'loss': 2.3886, 'learning_rate': 1.964999474992125e-05, 'epoch': 0.01}


  4%|▎         | 1000/28571 [17:30<8:02:43,  1.05s/it]

{'loss': 1.5815, 'learning_rate': 1.92999894998425e-05, 'epoch': 0.01}


  5%|▌         | 1500/28571 [25:48<7:23:28,  1.02it/s]

...

{'loss': 0.641, 'learning_rate': 3.9970599558993383e-07, 'epoch': 0.32}


100%|█████████▉| 28500/28571 [7:50:58<01:09,  1.02it/s]

{'loss': 0.6445, 'learning_rate': 4.9700745511182674e-08, 'epoch': 0.33}


100%|█████████▉| 28570/28571 [7:52:07<00:00,  1.02it/s]Saving model checkpoint to models/xlnet-smiles-s2000000-b70-l4\checkpoint-28570
Configuration saved in models/xlnet-smiles-s2000000-b70-l4\checkpoint-28570\config.json
Model weights saved in models/xlnet-smiles-s2000000-b70-l4\checkpoint-28570\pytorch_model.bin
100%|██████████| 28571/28571 [7:52:09<00:00,  1.29s/it]

ValueError: Trainer: evaluation requires an eval_dataset.